 # Collaborative Filtering and the Million Playlist Dataset

### Justin Moczynski, Gabe Pesco

## 1. Introduction

There are songs which are famous and appear on many playlists. There is also a significantly larger amount of songs which are unknown to the public eye. How do the lesser known tracks have a chance at becoming more popular? How can these songs be recommended to listeners in such a way that both popularity bias is reduced and the songs are suggested to an accurate set of listeners? In this project, we explored how to reduce the popularity bias in recommending music to listeners using a variation on a recommender system tasked with suggesting tracks to be added to a playlist.

## 2. Procedure

The dataset used for this project was the Million Playlist Dataset (MPD) provided by Spotify for their Recommender Systems Challenge. The first obstacle was importing the data and converting it from a JSON (JavaScript Object Notation) file to a npz file storing a compressed sparse row matrix.

### 2.1 Importing Data

<div class="alert alert-block alert-warning">
This code should only run once for the entire project because its purpose is to extract data and convert it into an npz file for use during the program. Once the npz file has been created, it does not need to be created multiple times since we hold a reference to these files.
</div>

First, the following libraries were imported to use in the data conversion process:

In [1]:
import json
import os
import sys
import numpy as np
import implicit
import scipy.sparse
import concurrent.futures
import threading
import time
from sklearn import *
from random import *

#### 2.1.1 Importing Test Data

After this, we began importing the data from the .json files. In order to do this, we used file scanners to read the contents of each file and dictionary, list, and set data structures to contain the scanned contents.

In [ ]:
path = "Data\\data_big\\"
files = os.listdir(path)
master_playlists = list()
master_songs_set = set()
master_songs_records = list()

def process_file(file):
    start_time = time.time()
    file_row = files.index(file)
    file_contents_json = open(path + file)
    file_contents = json.load(file_contents_json)
    playlists_in_file = file_contents['playlists']

    for playlist in playlists_in_file:
        playlist_name = dict(playlist)['name']
        songs = list()
        
        for song in dict(playlist)['tracks']:
            song_name = dict(song)['track_name']
            songs.append(song_name)
            master_songs_set.add(song_name)
            master_songs_records.append((song_name, playlist_name))
            
        master_playlists.append(playlist_name)

    del playlists_in_file
    print(str(file_row) + "," + time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)) + "\t")

def process_all_files(file_list):
    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=500) as executor:
        executor.map(process_file, file_list)
    print("all files processed in " + time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)) + "\t(" + str(len(master_playlists)) + " playlists, " + str(len(master_songs_set)) + " songs)")

process_all_files(files)

After extracting all of the data from the file, we create a sparse matrix with $m$ rows and $n$ columns where $m$ is the number of playlists and $n$ is the number of songs. This matrix is a sparse matrix because there are very few entries in the matrix which contain nonzero values.

In [ ]:
master_songs_set_list = list(master_songs_set)
master_songs_vector = np.asarray(master_songs_set_list)
m = len(master_playlists)
n = len(master_songs_set_list)
matrix = scipy.sparse.dok_matrix((m,n), dtype=int)

start_time_total = time.time()
counter = 0
for record in master_songs_records:
    start_time = time.time()
    matrix[master_playlists.index(record[1]),master_songs_set_list.index(record[0])] = 1
print("all playlists processed in " + time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time_total)))

del master_songs_set
del master_songs_set_list
del master_songs_vector

After creating the sparse matrix, we convert it from a DOK (dictionary of keys) matrix to a CSC (compressed sparse column) matrix in order to create an npz file from the data.

In [ ]:
file_save_name = "data_train.npz"
sparse_matrix = scipy.sparse.csr_matrix(matrix)
scipy.sparse.save_npz(file_save_name, sparse_matrix, "int32")


 The npz file is then reloaded into the program for further use.

In [ ]:
sparse_matrix = scipy.sparse.load_npz(file_save_name)
print(sparse_matrix.shape)

### 2.2 Penalizing for Popularity

In this project, we explored the introduction of a tunable hyperparameter in order to penalize songs for greater popularity. We defined a function which takes the parameters $X$ (the data matrix), and $\zeta$ (zeta, the hyperparameter) and scales the elements of the data matrix to penalize for popularity. We used the following formula to as a scale:
$$z = \frac{1-\zeta}{\zeta}$$
We also restricted $\zeta$ to $\zeta \neq 0$, so if $\zeta = 0$, then the function does not scale the matrix and returns the original matrix $X$.

In [ ]:
def popularityScale(X, zeta):
    #frequencies of each track
    freqs = np.sum(X, axis = 0)
    #popularity scaling term is the nth root over the original
    z = (1-zeta)/zeta
    #scalar to make it so the most popular song is populated with 1s 
    scalar = np.reciprocal(np.amax(freqs)**z)
    #applying frequency scaling to the matrix
    return X * (scalar * (freqs**z))

We converted the sparse matrix back into a dense matrix in order to run the popularityScale function on the imported data. The data was then converted back to a sparse matrix for memory conservation.

In [ ]:
print("size of sparse_matrix:", np.shape(sparse_matrix))
dense_matrix = scipy.sparse.dok_matrix.toarray(sparse_matrix)
scaled_matrix = scipy.sparse.dok_matrix(popularityScale(dense_matrix, 2))
print("size of scaled_matrix:", np.shape(scaled_matrix))

### 2.3 Running Alternating Least Squares

We used the scikit-learn library to run the Alternating Least Squares learning algorithm on the imported data.

In [6]:
def runTests():
    users=100
    n=5
    model = implicit.als.AlternatingLeastSquares(factors=224, use_gpu=False)
    raw = scipy.sparse.load_npz("Data\data10.npz")
    songlist = np.zeros(500)
    rawsonglist = np.zeros(500)
    scorelist = np.zeros(500)
    rawscorelist = np.zeros(500)
    for matrixes in range(10,21):
        string = "Data\data"+str(matrixes)
        string+=".npz"
        mat = scipy.sparse.load_npz(string)
        model.fit(mat.T)

        for row in range(users):
            recs = model.recommend(row, mat, N=n, recalculate_user=True)
            rawrecs = model.recommend(row, raw, N=n, recalculate_user=True)
            for i in range(n):
                rawsonglist[n*row+i]=rawrecs[i][0]
                rawscorelist[n*row+i]=rawrecs[i][1]
                songlist[n*row+i]=recs[i][0]
                scorelist[n*row+i]=recs[i][1]
        if(matrixes==10):
            baseline = np.average(scorelist)
            print("Baseline score:",baseline)
        print("Zeta parameter:",matrixes/10)
        print("number of unique songs recommended:",np.size(np.unique(songlist)))
        print("baseline - raw avg:",baseline-np.average(rawscorelist))
        print("error:",100*((baseline-np.average(rawscorelist))/baseline),"%")

In [7]:
runTests()

100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:07<00:00,  2.11it/s]


Baseline score: 0.27471219559
Zeta parameter: 1.0
number of unique songs recommended: 357
baseline - raw avg: 0.0
error: 0.0 %


100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:07<00:00,  1.87it/s]


Zeta parameter: 1.1
number of unique songs recommended: 380
baseline - raw avg: -0.00385210696515
error: -1.40223369293 %


100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:07<00:00,  1.93it/s]


...

Zeta parameter: 1.8
number of unique songs recommended: 432
baseline - raw avg: 0.0302018116404
error: 10.9939828392 %


100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:07<00:00,  1.91it/s]


Zeta parameter: 1.9
number of unique songs recommended: 432
baseline - raw avg: 0.0346597994064
error: 12.6167676436 %


100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:08<00:00,  1.77it/s]


Zeta parameter: 2.0
number of unique songs recommended: 435
baseline - raw avg: 0.038843041289
error: 14.1395401852 %


## 3. Conclusions

### 3.1 Potential Sources of Error

This project required a complex implementation. There are many potential sources of error, including the data importing process and inaccuracy in the model due to the amount of data used for testing. The dip below zero error in our testing (indicating that the recommendations of the matrix with a value of 1.1 used for regularization)

### 3.2 Future

In the future, we would like to be able to import the entire Million Playlist Dataset into a sparse matrix once. In addition, we would like to continue experimenting with the induced hyperparameter.